# 2.1 Wczytywanie sekwencji obrazów

Zadanie 2.1

In [1]:
import cv2
import numpy as np

In [2]:

for i in range(300,1100, 3):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    cv2.imshow("I",I)
    cv2.waitKey(10)
cv2.destroyAllWindows()

# 2.2 Odejmowanie ramek i binaryzacja

In [3]:
def I_gray(filename):
    I = cv2.imread(filename)
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    return I

In [4]:
def I_original(filename):
    I = cv2.imread(filename)
    return I

In [5]:
def I_diff_and_thresh(I_prev, I):
    I_abs_diff = cv2.absdiff(I_prev, I)
    (T, I_thresh) = cv2.threshold(I_abs_diff,20,255,cv2.THRESH_BINARY)
    return I_thresh

In [6]:
I_previous = I_gray('pedestrian/input/in%06d.jpg' % 299)
for i in range(300,1100):
    I = I_gray('pedestrian/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(I_previous, I)

    cv2.imshow("I_result", I_result)
    cv2.waitKey(10)
    I_previous = I
cv2.destroyAllWindows()

# 2.3 Operacje morfologiczne

In [7]:
def I__median_and_morfology(I):
    I_median = cv2.medianBlur(I, 3)
    I_median = cv2.medianBlur(I_median, 3)

    kernel = np.ones((5,5),np.uint8)
    # I_result = cv2.erode(I_median,kernel)
    I_result = cv2.dilate(I_median,kernel)
    I_result = cv2.dilate(I_median,kernel)


    return I_result

In [8]:
I_previous = I_gray('pedestrian/input/in%06d.jpg' % 299)
for i in range(300,1100):
    I = I_gray('pedestrian/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(I_previous, I)
    I_result = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result)
    cv2.waitKey(10)
    I_previous = I
cv2.destroyAllWindows()

# 2.4 Indeksacja i prosta analiza

In [8]:
def I_indexation(I_input, I):
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(I)

    I_VIS = I_input # copy of the input image
    if (stats.shape[0] > 1): # are there any objects
        tab = stats[1:,4] # 4 columns without first element
        pi = np.argmax( tab )# finding the index of the largest item
        pi = pi + 1 # increment because we want the index in stats, not in tab
        # drawing a bbox
        cv2.rectangle(I_VIS,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],
        stats[pi,1]+stats[pi,3]),(255,0,0),2)
        cv2.putText(I_VIS,"%f" % stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))
        cv2.putText(I_VIS,"%d" %pi,(int(centroids[pi,0]),int(centroids[pi,1])),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
    return I_VIS, np.uint8(labels / retval * 255)

In [10]:
I_previous = I_gray('pedestrian/input/in%06d.jpg' % 299)
for i in range(300,1100):
    I = I_gray('pedestrian/input/in%06d.jpg' % i)
    I_color = I_original('pedestrian/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(I_previous, I)
    I_result = I__median_and_morfology(I_result)
    I_result, tmp = I_indexation(I_color, I_result)
    cv2.imshow("I_result", I_result)
    cv2.waitKey(10)
    I_previous = I
cv2.destroyAllWindows()

# 2.5 Ewaluacja wyników detekcji obiektów pierwszoplanowych

In [10]:
TP, TN, FP, FN = 0, 0, 0, 0

I_previous = I_gray('pedestrian/input/in%06d.jpg' % 299)
for i in range(300,1100):
    I = I_gray('pedestrian/input/in%06d.jpg' % i)
    I_ref = I_gray('pedestrian/groundtruth/gt%06d.png' % i)
    I_color = I_original('pedestrian/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(I_previous, I)
    cv2.imshow("I_diff", I_result)

    I_result = I__median_and_morfology(I_result)
    cv2.imshow("I_morfology", I_result)

    I_label, tmp = I_indexation(I_color, I_result)
    cv2.imshow("I_result", I_label)
    cv2.waitKey(10)

    f = open('pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
    I_previous = I

cv2.destroyAllWindows()


P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

In [458]:
F1

0.6538007766213381

## Highway

In [7]:
TP, TN, FP, FN = 0, 0, 0, 0

I_previous = I_gray('highway/input/in%06d.jpg' % 299)
for i in range(470,1701):
    I = I_gray('highway/input/in%06d.jpg' % i)
    I_ref = I_gray('highway/groundtruth/gt%06d.png' % i)
    I_color = I_original('highway/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(I_previous, I)
    I_result = I__median_and_morfology(I_result)
    frame, indexed = I_indexation(I_color, I_result)

    cv2.imshow("I_result", frame)
    cv2.waitKey(10)

    f = open('highway/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
    I_previous = I

cv2.destroyAllWindows()


P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

KeyboardInterrupt: 

In [432]:
F1

0.785038049602141

## OFFICE

In [435]:
TP, TN, FP, FN = 0, 0, 0, 0

I_previous = I_gray('office/input/in%06d.jpg' % 299)
for i in range(570,2051):
    I = I_gray('office/input/in%06d.jpg' % i)
    I_ref = I_gray('office/groundtruth/gt%06d.png' % i)
    I_color = I_original('office/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(I_previous, I)
    I_result = I__median_and_morfology(I_result)
    frame, indexed = I_indexation(I_color, I_result)

    cv2.imshow("I_result", frame)
    cv2.waitKey(10)

    f = open('office/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
    I_previous = I

cv2.destroyAllWindows()


P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

In [434]:
F1

0.24917476552577006